In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from datetime import datetime, timedelta

In [2]:
user_data_path = "../result/user_data.json"
food_data_path = "../codes/food_data.json"
tracker_memory_path = "../result/tracker_memory.json"

In [3]:
# Load data
with open(user_data_path, "r") as f:
    user_data = json.load(f)

with open(food_data_path, "r") as f:
    food_data = json.load(f)

with open(tracker_memory_path, "r") as f:
    tracker_memory = json.load(f)


In [4]:
# Mapping makanan ke data nutrisinya
food_nutrition_map = {food["name"]: food for food in food_data}

In [5]:
# Mapping makanan ke data nutrisinya
food_nutrition_map = {food["name"]: food for food in food_data}

# Fungsi untuk menghitung kebutuhan kalori berdasarkan BMI
def calculate_daily_calories(weight, height, gender, activity_level):
    bmi = weight / ((height / 100) ** 2)  # BMI = berat (kg) / tinggi (m)^2
    base_calories = 2000 if gender == "male" else 1800

    # Penyesuaian berdasarkan tingkat aktivitas
    activity_factor = {
        "sedentary": 1.2,
        "lightly_active": 1.375,
        "moderately_active": 1.55,
        "very_active": 1.725
    }.get(activity_level, 1.2)

    return base_calories * activity_factor

# Ambil log makanan selama 3 bulan terakhir
three_months_ago = datetime.now() - timedelta(days=90)
valid_user_names = {user["user_name"] for user in user_data}

# Dataset untuk rekomendasi
X = []
y = []

for date, logs in tracker_memory.items():
    for log in logs:
        user_name = log["user_name"]

        # Validasi user
        if user_name not in valid_user_names:
            continue

        user_info = next(user for user in user_data if user["user_name"] == user_name)

        # Ambil tinggi dan berat badan
        weight = float(user_info["weight"].split()[0])  # Contoh: "59 kg"
        height = float(user_info["tall"].split()[0])  # Contoh: "156 cm"

        # Hitung kebutuhan kalori harian
        daily_calories = calculate_daily_calories(weight, height, user_info["gender"], user_info["ActiveLevel"])

        # Fitur input
        input_features = [
            weight,  # Berat user
            height,  # Tinggi user
            daily_calories,  # Kebutuhan kalori harian
            log["calories"],  # Kalori konsumsi terakhir
            log["protein"],  # Protein konsumsi terakhir
            log["fat"],  # Lemak konsumsi terakhir
            log["carbs"]  # Karbohidrat konsumsi terakhir
        ]
        X.append(input_features)

        # Label output (weight_in_grams dari log terakhir)
        y.append(log["weight_in_grams"])

# Konversi ke numpy array
X = np.array(X)
y = np.array(y)

In [6]:
# Konversi ke numpy array
X = np.array(X)
y = np.array(y)

print(f"Panjang X: {len(X)}")
print(f"Panjang y: {len(y)}")
for user_name, user_logs in tracker_memory.items():
    user_info = next((user for user in user_data if user["user_name"] == user_name), None)
    if not user_info:
        print(f"User {user_name} tidak ditemukan di user_data.json")
        continue

    weight = float(user_info["weight"].split()[0])
    tall = float(user_info["tall"].split()[0])

    for date, logs in user_logs.items():
        for log in logs:
            food_info = food_nutrition_map.get(log["food_name"])
            if not food_info:
                print(f"Makanan {log['food_name']} tidak ditemukan di food_data.json")
                continue

            input_features = [
                weight, tall, food_info["calories"], food_info["proteins"],
                food_info["fat"], food_info["carbohydrate"],
                log["calories"], log["protein"], log["fat"], log["carbs"]
            ]
            print(f"Menambahkan data input: {input_features}")
            X.append(input_features)
            y.append(log["weight_in_grams"])


Panjang X: 2000
Panjang y: 2000
User 2024-03-18 tidak ditemukan di user_data.json
User 2024-05-15 tidak ditemukan di user_data.json
User 2024-09-05 tidak ditemukan di user_data.json
User 2024-02-11 tidak ditemukan di user_data.json
User 2024-05-10 tidak ditemukan di user_data.json
User 2024-02-23 tidak ditemukan di user_data.json
User 2024-07-17 tidak ditemukan di user_data.json
User 2024-01-18 tidak ditemukan di user_data.json
User 2024-01-12 tidak ditemukan di user_data.json
User 2024-04-08 tidak ditemukan di user_data.json
User 2024-08-19 tidak ditemukan di user_data.json
User 2024-09-07 tidak ditemukan di user_data.json
User 2024-01-26 tidak ditemukan di user_data.json
User 2024-11-07 tidak ditemukan di user_data.json
User 2024-08-27 tidak ditemukan di user_data.json
User 2024-10-14 tidak ditemukan di user_data.json
User 2024-03-30 tidak ditemukan di user_data.json
User 2024-01-17 tidak ditemukan di user_data.json
User 2024-09-29 tidak ditemukan di user_data.json
User 2024-09-12 ti

In [7]:
# Split data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bangun model TensorFlow
def build_recommendation_model(input_dim):
    model = Sequential([
        Dense(64, activation="relu", input_dim=input_dim),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(1)  # Output: Rekomendasi porsi (gram)
    ])
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
    return model

In [9]:
#train
model = build_recommendation_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test))


Epoch 1/100
200/200 [==============================] - 2s 5ms/step - loss: 8413.6533 - mae: 68.9241 - val_loss: 4067.1831 - val_mae: 53.3738
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 4625.8862 - mae: 57.0746 - val_loss: 4760.3569 - val_mae: 55.8383
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 4585.7729 - mae: 56.9981 - val_loss: 3887.8884 - val_mae: 52.5892
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 4502.5586 - mae: 56.3804 - val_loss: 3916.6953 - val_mae: 52.9360
Epoch 5/100
200/200 [==============================] - 1s 4ms/step - loss: 4495.1206 - mae: 56.2723 - val_loss: 3933.7219 - val_mae: 53.3060
Epoch 6/100
200/200 [==============================] - 1s 4ms/step - loss: 4537.5396 - mae: 56.7969 - val_loss: 3904.8035 - val_mae: 52.9087
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 4517.6406 - mae: 55.9761 - val_loss: 4105.3794 - val_mae: 53.8572
Epoch 8/100
2

In [10]:
print("Model Summary:")
model.summary()

print("\nDimensi Input Model:", model.input_shape)
print("Dimensi Output Model:", model.output_shape)

print("\nDimensi Data Input (X):", X.shape)
print("Dimensi Data Output (y):", y.shape)

Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,137
Trainable params: 3,137
Non-trainable params: 0
_________________________________________________________________

Dimensi Input Model: (None, 7)
Dimensi Output Model: (None, 1)

Dimensi Data Input (X): (2000, 7)
Dimensi Data Output (y): (2000,)


In [54]:
# Simpan model ke file
model.save("result/recommendation_model")

print("Model berhasil dilatih dan disimpan!")

INFO:tensorflow:Assets written to: result/recommendation_model\assets
Model berhasil dilatih dan disimpan!


In [56]:
def recommend_food(user_name, time_of_day):
    user_info = next(user for user in user_data if user["user_name"] == user_name)

    # Ambil tinggi dan berat badan
    weight = float(user_info["weight"].split()[0])
    height = float(user_info["tall"].split()[0])
    daily_calories = calculate_daily_calories(weight, height, user_info["gender"], user_info["ActiveLevel"])

    # Alokasi kalori berdasarkan waktu makan
    if time_of_day == "breakfast":
        calorie_allocation = daily_calories * 0.22
    elif time_of_day == "lunch":
        calorie_allocation = daily_calories * 0.31
    elif time_of_day == "dinner":
        calorie_allocation = daily_calories * 0.35
    else:
        raise ValueError("Waktu makan tidak valid (breakfast, lunch, dinner)")

    recommendations = []
    for food_name, food_info in food_nutrition_map.items():
        # Prediksi berat makanan
        portion_weight = model.predict(np.array([[weight, height, daily_calories, 0, 0, 0, 0]]))
        portion_weight = portion_weight[0][0]

        if portion_weight < 50 or portion_weight > 500:
            continue

        recommendations.append({
            "food_name": food_name,
            "portion_weight": round(portion_weight, 1),
            "calories": round((food_info["calories"] / 100) * portion_weight, 1),
            "protein": round((food_info["proteins"] / 100) * portion_weight, 1),
            "fat": round((food_info["fat"] / 100) * portion_weight, 1),
            "carbs": round((food_info["carbohydrate"] / 100) * portion_weight, 1)
        })

    # Prioritaskan makanan berdasarkan protein
    recommendations.sort(key=lambda x: (-x["protein"], -x["calories"]))

    return recommendations

# Contoh rekomendasi
recommended_foods = recommend_food("User700", "lunch")
print(recommended_foods)


1/1 [==============================] - 0s 31ms/step
[{'food_name': 'Kerupuk Kulit kerbau', 'portion_weight': 84.0, 'calories': 354.7, 'protein': 69.8, 'fat': 3.4, 'carbs': 0.0}, {'food_name': 'Dendeng mujahir goreng', 'portion_weight': 84.0, 'calories': 502.6, 'protein': 62.4, 'fat': 22.6, 'carbs': 7.7}, {'food_name': 'Ikan Mujair dendeng goreng', 'portion_weight': 84.0, 'calories': 502.6, 'protein': 62.4, 'fat': 22.6, 'carbs': 7.7}, {'food_name': 'Ikan kayu kering', 'portion_weight': 84.0, 'calories': 253.8, 'protein': 59.4, 'fat': 1.6, 'carbs': 0.3}, {'food_name': 'Ikan Teri Kering tawar', 'portion_weight': 84.0, 'calories': 278.2, 'protein': 57.7, 'fat': 3.5, 'carbs': 0.0}, {'food_name': 'Ikan Mujair dendeng mentah', 'portion_weight': 84.0, 'calories': 489.1, 'protein': 57.4, 'fat': 12.8, 'carbs': 31.3}, {'food_name': 'Udang kering mentah', 'portion_weight': 84.0, 'calories': 233.6, 'protein': 52.6, 'fat': 3.4, 'carbs': 1.8}, {'food_name': 'Udang kering', 'portion_weight': 84.0, 'ca

In [61]:
import os

# Path untuk menyimpan model
model_save_path = "D:/Bangkit 2024/capstone project/result/model"

# Buat direktori jika belum ada
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Simpan model ke file .h5
model.save(os.path.join(model_save_path, "recommendation_model.h5"))
print(f"Model berhasil disimpan ke {os.path.join(model_save_path, 'recommendation_model.h5')}")


Model berhasil disimpan ke D:/Bangkit 2024/capstone project/result/model\recommendation_model.h5


In [ ]:
import tensorflowjs as tfjs

# Path file .h5 dan folder output TensorFlow.js
h5_model_path = "result/model/recommendation_model.h5"
tfjs_output_path = "result/model/tfjs"

# Konversi model ke TensorFlow.js
tfjs.converters.save_keras_model(tf.keras.models.load_model(h5_model_path), tfjs_output_path)
print(f"Model berhasil dikonversi ke TensorFlow.js di folder {tfjs_output_path}")


ModuleNotFoundError: No module named 'tensorflowjs'